### Testing Start

In [1]:
%pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,  # 对图像的每个像素值均乘以该数值后再做偏移，实现归一化
    rotation_range= 20,  # 随机旋转的度数范围
    width_shift_range=0.1,  # 图片宽度上随机移动的范围，这里的0.2指最多移动图片宽度的20%
    height_shift_range=0.1,  # 图片高度上随机移动的范围，这里的0.2指最多移动图片高度的20%
    shear_range=0.15,  # 剪切强度（逆时针方向的剪切变换角度）
    zoom_range=0.15,  # 随机缩放的幅度
    horizontal_flip=True,  # 随机将图片水平翻转
    fill_mode='nearest',  # 当进行变换时，新出现的像素点如何填充
    validation_split=0.2)  # 从训练数据中划分出20%的数据作为验证数据


In [ ]:
train_generator = train_datagen.flow_from_directory(
    'images',  # 训练数据文件夹路径
    target_size=(100, 100),  # 将所有图片大小调整为100x100
    batch_size=32,  # 每批数据的大小
    class_mode='categorical',  # 由于我们进行多类别分类，使用"分类"模式
    subset='training')  # 指定这是用于训练的数据

validation_generator = train_datagen.flow_from_directory(
    'images',  # 训练数据文件夹路径，同样用于验证数据的提取
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # 指定这是用于验证的数据


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),# 32 filters, 3x3 kernel
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),# 64 filters, 3x3 kernel

    MaxPooling2D(2, 2),# 2x2 pool
    Conv2D(128, (3, 3), activation='relu'),# 128 filters, 3x3 kernel

    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),

    Dense(9, activation='softmax')  # 9 neurons for 9 classes
])


In [ ]:
optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
# Calculate the appropriate steps per epoch
train_image_count = len(train_generator.filenames)
validation_image_count = len(validation_generator.filenames)

train_image_count, validation_image_count

# steps_per_epoch = train_image_count // train_generator.batch_size
# validation_steps = validation_image_count // validation_generator.batch_size


In [ ]:
from PIL import Image

history = model.fit(
    train_generator,
    steps_per_epoch=12,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=8
)


In [ ]:
import matplotlib.pyplot as plt#evaluating the model

# 绘制训练和验证的准确率值
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# 绘制训练和验证的损失值
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def prepare_image(image_path):
    img = load_img(image_path, target_size=(100, 100))  #load image, resize to 100x100, default color mode is RGB
    img_array = img_to_array(img)  # convert image to numpy array
    img_array = img_array / 255.0  # normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # expand dimension to match the model's expected input
    return img_array# Replace with your image path, e.g. 'image.png'

# 替换为你的图片路径
image_path = 'image.png'# Prepare the image
prepared_img = prepare_image(image_path)# Get the model's prediction


In [ ]:
predictions = model.predict(prepared_img)# Get the predicted class index
predicted_class = np.argmax(predictions[0])  #g
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # 反转类别索引和类别名称的字典
predicted_class_name = class_labels[predicted_class]  # 获取类别名称
print("Predicted class:", predicted_class_name)


# Save the model

In [ ]:
model.save('Model/model.h5')

In [ ]:
from keras.models import load_model

My_model = load_model('Model/model.h5')


In [23]:
image_path = 'image.png'
prepared_img = prepare_image(image_path)
predictions = My_model.predict(prepared_img)# Get the predicted class index
predicted_class = np.argmax(predictions[0])  #g
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # 反转类别索引和类别名称的字典
predicted_class_name = class_labels[predicted_class]  # 获取类别名称
print("Predicted class:", predicted_class_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: strawberry fruit


### Testing End